In [ ]:
import pandas as pd
import numpy as np
import torch
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# df.shape

In [ ]:
df = pd.read_csv('spam.csv')
df =df.iloc[:1200, :]
X = df['text']
y = df['class'].replace({'spam':0,'ham':1})

sentences = X.tolist()
target = y.tolist()

word_list = " ".join(sentences).split()
word_list = list(set(word_list))+['spam','ham']
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

<ipython-input-81-d99250409226>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = df['class'].replace({'spam':0,'ham':1})


In [ ]:
pdd = max(list(map(len,sentences)))
pdd

910

In [ ]:
# x = [torch.nn.functional.pad(torch.tensor([word_dict[j] for j in i.split()],dtype=torch.float64),(pdd-len([word_dict[j] for j in i.split()]),0)) for i in sentences ]


In [ ]:
#####################################gemini
import torch

# Assuming x, pdd, word_dict, and sentences are defined as in your original code

# Modify the padding to pad on the right side only
x = [torch.nn.functional.pad(torch.tensor([word_dict[j] for j in i.split()], dtype=torch.int64), (0, pdd - len([word_dict[j] for j in i.split()]))) for i in sentences]


x_tensor = torch.stack(x) # Use torch.stack to combine tensors of the same shape into a single tensor

# Now x_tensor should be a valid PyTorch tensor
print(x_tensor)

tensor([[1789, 2944,  672,  ...,    0,    0,    0],
        [ 128, 4183, 3611,  ...,    0,    0,    0],
        [3969, 5117, 4263,  ...,    0,    0,    0],
        ...,
        [1964,   52, 1254,  ...,    0,    0,    0],
        [ 640, 1194, 1132,  ...,    0,    0,    0],
        [5488, 4607,  883,  ...,    0,    0,    0]])


In [ ]:

input = torch.stack(x)
output = torch.from_numpy(np.array(y))
input.shape,output.shape

(torch.Size([1200, 910]), torch.Size([1200]))

In [ ]:
input.shape , output.shape

(torch.Size([1200, 910]), torch.Size([1200]))

In [ ]:
# 1. INPUT  OR OUTPUT TENSOR CONNECTION EACH OTHER
from torch.utils.data import TensorDataset
train_ds = TensorDataset(input , output)
train_ds[0:3]

(tensor([[1789, 2944,  672,  ...,    0,    0,    0],
         [ 128, 4183, 3611,  ...,    0,    0,    0],
         [3969, 5117, 4263,  ...,    0,    0,    0]]),
 tensor([1, 1, 0]))

In [ ]:
# 2. BATCH DATA USING DATA LOADER

from torch.utils.data import DataLoader
bs = 100
train_loader = DataLoader(train_ds, batch_size = bs, shuffle = True)
for xy, yb in train_loader:
    print(xy)
    print(yb)
    break


tensor([[3850, 1194, 3684,  ...,    0,    0,    0],
        [3850, 2719, 4992,  ...,    0,    0,    0],
        [4046, 5045,  844,  ...,    0,    0,    0],
        ...,
        [2088,  445, 3490,  ...,    0,    0,    0],
        [  22,  599, 3528,  ...,    0,    0,    0],
        [2841,  522, 1947,  ...,    0,    0,    0]])
tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1])


In [ ]:
# # 3. MODEL CREATION
n_step = 2 # number of steps, n-1 in paper
n_hidden = 2 # number of hidden size, h in paper
m = 910 # embedding size, m in paper

class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.C = nn.Embedding(n_class, m) # n_class as the vocabulary size  and m as the embedding dimension.
    self.r = torch.nn.RNN(m,32,batch_first=True) # hidden size of 32. # batch dimension as the first dimension
    self.H = nn.Linear( 32, 1, bias=False) #linear layer with an input size of 32 output size of 1.bias=False means this layer does not include a bias term
  def forward(self,X):
        X = self.C(X) # X : [batch_size, n_step, m]
        X,_ = self.r(X) # X : [batch_size, n_step, m]
        # print(X[:,-1,:])
        X = X[:,-1,:]
        a = self.H(X) # X : [batch_size, n_step, m]
        return torch.nn.functional.sigmoid(a)

model = Model()
model(xy)

tensor([[0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0.6509],
        [0

In [ ]:
# 4. LOSS CALCULATION

criterion = nn.BCELoss() # Binary Cross Entropy Loss
"""
Binary Cross Entropy Loss (BCE Loss) is a loss function commonly used in binary classification problems where the goal
 is to predict one of two possible classes (e.g., 0 or 1). This loss function measures the difference between
 the predicted probabilities and the actual binary labels.
"""

In [ ]:
# 5. OPTIMIZER
optimizer = optim.Adam(model.parameters(), lr=0.001)
"""
Adaptive Learning Rates: Adam computes individual adaptive learning rates for different parameters from estimates of first and second moments of the gradients. This means that parameters that have been updated less frequently receive larger updates, while parameters that have been updated frequently receive smaller updates.
Bias Correction: Adam includes bias-correction terms to compensate for the initialization of the first and second moment estimates. This helps stabilize the learning process, especially in the early stages of training.
Benefits:

Adam generally converges faster than traditional optimizers like Stochastic Gradient Descent (SGD), especially for problems with large datasets and/or high-dimensional parameter spaces.
It is less sensitive to the choice of hyperparameters compared to other optimizers."""

In [ ]:
# # 6. EPOCHES


In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    # correct = 0
    total = 0
    correct = 0
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_acc = 0
        # Train with batches of data
        for xb,yb in train_dl:
          # try:
            # 1. Generate predictions
            pred = model(xb)
            # print(pred.flatten() , yb)
            # 2. Calculate loss
            # asi = torch.argmax(pred,axis=1)
            # print(asi.type(torch.float32), yb)
            # break
            loss = loss_fn(pred.flatten(), yb.type(torch.float32))


            # y_pred_tag = torch.round(pred)
            # correct_results_sum = (y_pred_tag == yb).sum().float()
            # acc = correct_results_sum/yb.shape[0]
            # acc = torch.round(acc * 100)

            # 3. Compute gradients
            loss.backward() # back propogation

            # 4. Update parameters using gradients
            opt.step() # it is update the weight for gradients

            # 5. Reset the gradients to zero
            opt.zero_grad() # for the zero the gradient bez new calculation of the neuron

            epoch_loss += loss.item()

        # Print the progress
        # if (epoch+1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

          # except:
          #   return xb

d = fit(50, model, criterion, optimizer, train_loader)
d

Epoch [1/50], Loss: 0.4227
Epoch [1/50], Loss: 0.4053
Epoch [1/50], Loss: 0.4227
Epoch [1/50], Loss: 0.4926
Epoch [1/50], Loss: 0.4751
Epoch [1/50], Loss: 0.2835
Epoch [1/50], Loss: 0.4750
Epoch [1/50], Loss: 0.4924
Epoch [1/50], Loss: 0.4401
Epoch [1/50], Loss: 0.3360
Epoch [1/50], Loss: 0.4574
Epoch [1/50], Loss: 0.3880
Epoch [2/50], Loss: 0.5094
Epoch [2/50], Loss: 0.3362
Epoch [2/50], Loss: 0.4227
Epoch [2/50], Loss: 0.3534
Epoch [2/50], Loss: 0.4922
Epoch [2/50], Loss: 0.4053
Epoch [2/50], Loss: 0.4401
Epoch [2/50], Loss: 0.4749
Epoch [2/50], Loss: 0.3706
Epoch [2/50], Loss: 0.4227
Epoch [2/50], Loss: 0.5095
Epoch [2/50], Loss: 0.3534
Epoch [3/50], Loss: 0.4747
Epoch [3/50], Loss: 0.3363
Epoch [3/50], Loss: 0.4573
Epoch [3/50], Loss: 0.3881
Epoch [3/50], Loss: 0.4920
Epoch [3/50], Loss: 0.3535
Epoch [3/50], Loss: 0.4227
Epoch [3/50], Loss: 0.4401
Epoch [3/50], Loss: 0.3880
Epoch [3/50], Loss: 0.3532
Epoch [3/50], Loss: 0.6145
Epoch [3/50], Loss: 0.3706
Epoch [4/50], Loss: 0.4401
E

In [ ]:
# sen = 'Aaooooright are you at work?'
sen =df['text'][ind]
a = [word_dict[i] for i in sen.split()]
a = torch.tensor(a, dtype=torch.int64)
testing = torch.nn.functional.pad(a, (0, pdd - len(a))) # 0: No padding is added to the left (or beginning) of the tensor
out = model(testing.unsqueeze(0)) # adds a new dimension at the beginning, so it represents a batch with one sample of three elements.
['spam' if out< 0.50 else 'ham']

['ham']

In [ ]:
ind = 127
print(df.iloc[ind,:])


class                       ham
text     Are you there in room.
Name: 127, dtype: object


In [ ]:
df.iloc[:200,0].value_counts()

,count
class,
ham,167
spam,33


In [ ]:
# THEORY
"""torch.nn.RNN is a class in PyTorch that implements a basic recurrent neural network (RNN) layer.
 RNNs are designed to process sequential data by maintaining a hidden state that captures information from previous time steps,
 making them suitable for tasks such as time series prediction, natural language processing (NLP), and more.
"""

"""torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None)
num_embeddings: The number of unique items in the input space. For example, if you're embedding words in a vocabulary of 10,000 unique words, this value would be 10,000.

embedding_dim: The dimensionality of the embedding vectors, which defines the size of the vector space in which the items will be represented. For instance, setting embedding_dim=50 would represent each word as a 50-dimensional vector.

padding_idx: (Optional) Specifies an index to be treated as a padding token. When a tensor containing this index is passed, it will be ignored in gradient calculations (useful in NLP when handling variable-length sequences).


#RNN
torch.nn.RNN is a class in PyTorch that implements a basic recurrent neural network (RNN) layer.
RNNs are designed to process sequential data by maintaining a hidden state that captures information from previous time steps,
making them suitable for tasks such as time series prediction, natural language processing (NLP), and more.

The class takes several parameters:
input_size: The number of expected features in the input. For example, if you are working with word embeddings of size 50, input_size would be 50.
hidden_size: The number of features in the hidden state. This determines the size of the hidden state vector that the RNN will produce.
num_layers: The number of recurrent layers. A value of 1 means a single layer, while a value of 2 or more stacks additional RNN layers on top of each other.
nonlinearity: The activation function to use. The default is 'tanh', but you can also use 'relu' if desired.
bias: If True, adds a learnable bias to the output. Default is True.
batch_first: If True, then the input and output tensors are provided as (batch, seq, feature). If False, they are provided as (seq, batch, feature).

Purpose of a Linear Layer
Functionality: The primary purpose of a linear layer is to perform a linear transformation on the input data.
nn.Linear(32, 1, bias=False):

This initializes a linear layer with specific parameters:
32: The size of the input features. This means that the layer expects input tensors with 32 features. Each input sample should be a vector of size 32.
1: The size of the output features. This means that the layer will produce an output with a single feature. In many cases, this might represent a scalar value (like a prediction).
bias=False: This specifies that the layer should not include a bias term. In a typical linear layer, the output is calculated as:
output = WX+B
b is the bias vector. By setting bias=False, the model will not learn an additional bias term during training.

"""



